<a href="https://colab.research.google.com/github/shouvikcirca/LLMs/blob/gemini/RAG_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install google-generativeai
! pip install chromadb
! pip install langchain-google-genai
! pip install langchain
! pip install langchain-community

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.vectorstores import Chroma
from dotenv import load_dotenv
import os
from langchain_community.document_loaders.telegram import text_to_docs
from langchain.chains.combine_documents import create_stuff_documents_chain
import google.generativeai as genai

In [4]:
_= load_dotenv('drive/MyDrive/env')
api_key = os.environ['GEMINI_API_KEY']

In [5]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=api_key)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=api_key)

In [6]:
with open('drive/MyDrive/LLM/synthetic_data_cb.txt') as f:
    data = f.read().split('\n')

In [7]:
years = range(1960,2021)
qlist = []

for y in years:
     for i in data:
            q = i.replace('2021',str(y))
            q+='closing balance for {}'.format(y)
            nq = 'Question: {} ?'.format(q.split('?')[0]) + '\nAnswer: {}'.format(q.split('?')[1])
            qlist.append(nq)

In [34]:
qlist[5]

'Question: What was the balance at the end of last year ?\nAnswer: closing balance for 1960'

In [18]:
vectordb = Chroma.from_documents(
    collection_name = 'cb1',
    documents = text_to_docs(qlist),
    embedding = embeddings,
    persist_directory="drive/MyDrive/LLM/chroma_db"
)

In [15]:
# To delete contents of a vectorstore
# for id in vectordb.get()['ids']:
#   vectordb._collection.delete(ids=[id])

In [ ]:
vectordb.get()

In [20]:
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

In [21]:
template = """
You are a helpful assistant
{context}
Based on the examples given above, give the answer for the following
{input}
"""

In [22]:
prompt = PromptTemplate.from_template(template)

In [23]:
combine_docs_chain = create_stuff_documents_chain(llm, prompt)

In [24]:
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [29]:
response=retrieval_chain.invoke({"input":"Closing balance for 1967 ?"})

In [31]:
q2 = response["answer"]
q2

'closing balance for 1967'

In [39]:
identity_prompt_template = "Give me back as output whatever i have provided as input in this query \n {}"

In [56]:
def getClosingBalance(year:str):
  """ Takes in the year and returns closing balance for the given year """
  return "Closing balance for {} is Rs. 22500".format(year)

In [57]:
model = genai.GenerativeModel(
    model_name="gemini-pro",
    generation_config={'temperature':0.0},
    tools = [getClosingBalance]
)

In [58]:
chat = model.start_chat(enable_automatic_function_calling=True, history=[])

In [59]:
identity_prompt_template.format(q2)

'Give me back as output whatever i have provided as input in this query \n closing balance for 1967'

In [60]:
response = chat.send_message(identity_prompt_template.format(q2))

In [62]:
response.text

'Closing balance for 1967 is Rs. 22500'